In [ ]:
path = "1_marked.edf"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyedflib
import locale, pprint, time, calendar
import torch
import torch.nn as nn
import torch.nn.functional as F

читаем edf, число каналов, названия каналов, частоту дискретизации, метки

In [ ]:
import pyedflib
edf_file_path = path
edf_file = pyedflib.EdfReader(edf_file_path)


num_channels = edf_file.signals_in_file
channel_labels = edf_file.getSignalLabels()
sample_rate = edf_file.getSampleFrequency(0)

n_annotations = edf_file.annotations_in_file
annotations=[]
epi={}
for i in range(num_channels):
    epi[channel_labels[i]] = edf_file.readSignal(i)
    
annotation = edf_file.read_annotation()
edf_file.close()

In [ ]:
sample_rate

In [ ]:
num_channels

чтобы обозначить границы размеченной записи, указываем начало и конец в формате 'hh:mm:ss', получаем начало и конец в отсчётах с учетом частоты дискретизации

In [ ]:
def get_sec(time):
    h, m, s = time.split(':')
    return int(h)*3600+int(m)*60+int(s)

In [ ]:
int(get_sec('0:00:00')*sample_rate)

In [ ]:
fig, axs = plt.subplots(23, 1, figsize=(30,40), dpi=400)
keys_epi=list(epi.keys())
example=[]
start = int(get_sec('0:00:00')*sample_rate)
finish = int(get_sec('0:16:00')*sample_rate)
for i in range(len(keys_epi)):
    axs[i].plot(epi[keys_epi[i]][start:finish])
    axs[i].set_title(f'{channel_labels[i]}')
    example.append(epi[keys_epi[i]][start:finish])

Извлечение нужных меток

In [ ]:
len(annotation)

In [ ]:
annotation

формат записи - начало в отсчётах и длительность в секундах

In [ ]:
doc_annot = [[(x[0]//10000000)*int(sample_rate) + (x[0]%10000000)//20000 - start, float(x[1])] for x in annotation if ('F7' in str(x[2]) and x[1]!=b'' and ('EEG' not in str(x[2])))]

In [ ]:
doc_annot

In [ ]:
signal = example[10] #F7 channel

In [ ]:
doc_annot

приведение к маске

In [ ]:
doc_delin = [[int(delin[0]), int(delin[0]+int(delin[1]*sample_rate))] for delin in doc_annot ]

In [ ]:
delineation = np.zeros(finish-start)
for label in doc_delin:
    print(label)
    for tick in range(label[0], label[1]+1):
        delineation[tick]=1

In [ ]:
len(delineation)

In [ ]:
delineation[4600:4680]

In [ ]:
X=[]
Y=[]
for i in range((finish-start)//5000):
    X.append(signal[5000*i:5000*i+5000])
    Y.append(delineation[5000*i:5000*i+5000])

In [ ]:
i=-6
plt.figure(figsize=(15, 6), dpi=400)
plt.plot(X[i])
plt.plot(Y[i]*150)

In [ ]:
X_notempty=[]
Y_notempty=[]
for i in range(len(X)):
    if max(Y[i]) == 1:
        X_notempty.append(X[i])
        Y_notempty.append(Y[i])

In [ ]:
len(Y_notempty)

In [ ]:
# for i in range(len(X_notempty)):
#     plt.figure(figsize=(15,10))
#     plt.plot(X_notempty[i])
#     plt.plot(Y_notempty[i]*150)
#     plt.show()

In [ ]:
np.save('1X.npy', X_notempty)
np.save('1Y.npy', Y_notempty)